In [1]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data

In [2]:
#传统神经网络的问题：
  #权值多，计算量大，需要大量样本进行训练
#局部感受野：
  #通过感受野和权值共享减少需要训练参数的个数，每个神经元只链接到输入的一部分
#矩阵的卷积
  #对应各个元素相乘再相加，通过卷积核（相当于滤波器）进行采样，采用多个卷积核对同一个图片进行采样，然后得到不同的特征图
#池化
  #与卷积相似，采用的方式：1.选择某选定区域的最大值作为池化值，2.取选定区域的最大值作为池化值 3.随机选取
#对卷积的PADDING操作
  #1.SAME:可能给平面外补0，得到的平面与源平面的大小相同
  #2.VALID：不会超出平面，得到的平面比原来的平面小或者相同
  #e.g:28*28->经过两种方式得到的相同，2*3->S(2*2,f=2)->1*2  2*3->V(2*2,f=2)->1*1
#CNN的结构
  #图片->卷积层->池化层->卷积层->池化层->全链接层...->输出

#载入数据集,从网络中获取，使用one_hot来将标签变为0/1格式
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)


#每个批次大小
batch_size = 50
#计算一共有多少批次  //为整除
n_batch = mnist.train.num_examples // batch_size

#初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1) #生成截断正态分布
    return tf.Variable(initial)

#初始化偏置值
def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#卷积层二维
def conv2d(x,W):
    #x input tensor of shape  [batch,in_height,in_width,in_channels] 长 宽 通道数
    #W filter / kernel tensor of shape [filter_height,filter_witdth,in_channels,out_channels]
    #'strides[0] = strides[3] = 1',strides[1]代表x方向的步长，strides[2]代表y方向的步长
    #padding : A string from :'SAME','VALID'
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding="SAME")

#池化层
def max_pool_2x2(x):
    #ksize[1,x,y,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

#定义两个占位符
x = tf.placeholder(tf.float32,[None,784],name="x-input")
y = tf.placeholder(tf.float32,[None,10],name="y-input")

#改变x的格式为4D的向量[batch,in_height,in_width,in_channels]
x_image = tf.reshape(x,[-1,28,28,1])

#初始化第一个卷积层的权值和偏置
W_conv1 = weight_variable([5,5,1,32])  #5*5的采样窗口，32个卷积核从1个平面抽取特征
b_conv1 = bias_variable([32])  #每个卷积核一个偏置值

#把x_image和权值向量进行卷积，再加上偏置值,然后选用relu激活函数
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1) #进行max_pooling

#初始化第二个卷积层的权值和偏置
W_conv2 = weight_variable([5,5,32,64])  #5*5的采样窗口，64个卷积核从32个平面抽取特征
b_conv2 = bias_variable([64])  #每个卷积核一个偏置值

#把h_pool1和权值向量进行卷积，再加上偏置值,然后选用relu激活函数
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2) #进行max_pooling

#28*28的图片第一次卷积之后还是28*28，然后第一次池化后变为14*14
#第二次卷积后变为14*14，第二次池化后变为7*7
#操作之后得到64张7*7的平面

#初始化第一个全链接层的权值
W_fc1 = weight_variable([7*7*64,1024]) #上层有7*7*64个神经元，全链接层有1024个神经元
b_fc1 = bias_variable([1024])

#把池化层2扁平化为1维
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
#求第一个全链接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

#使用keep_prob表示神经元的输出概率
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

#初始化第二个全链接层的权值
W_fc2 = weight_variable([1024,10]) 
b_fc2 = bias_variable([10])

#计算输出
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

#交叉熵代价函数
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
#使用AdamOptimizer进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#比较两个参数是否相同，返回布尔型列表，用于检测训练效果,argmax函数用于求最大概率的位置（标签），返回以为张量中最大值所再位置
correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1)) 
#求准确率,cast将对比的布尔型转换为浮点型，然后求平均值
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):  #把所有的训练21次
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)   #把训练集中的图片循环一次
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})  #70%的神经元工作
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        print("Iter" + str(epoch)+ ",Test Accuracy" + str(acc))       

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input

KeyboardInterrupt: 